## Intro

A notebook is a good way to document code and share your thoughts with others (and with yourself in the future). Interestingly, *github displays notebooks beautifully*. By checking in this file to a public repository, github serves my blog. Since it is tracked in git, as I publish updates to it, all the history is maintained - pretty cool.

## Division Operators and Unicode

There is a division operator - ÷ - that is the same as "/" (almost). Julia is a new language that has embraced unicode. This is a huge difference from MATLAB and Python. I'm not comfortable with it yet. To enter the division symbol in this Julia notebook, you can type `\div<tab>`. As a "vi" user, I am now able to enter these in my favorite go-to editor, without having to copy and paste from another location - using the https://github.com/JuliaEditorSupport/julia-vim plugin. I also use "eclipse" - I haven't found out how to cleanly handle unicode there.

In [1]:
7÷2

3

Thanks to [Kristoffer Carlsson](https://kristofferc.github.io/) for pointing out that ÷ and / are NOT in fact the same (comment on [this thread](https://discourse.julialang.org/t/julia-is-a-very-different-and-interesting-language/10741/2)). 

In [2]:
?÷

"÷" can be typed by \div<tab>

search: ÷ .÷



```
div(x, y)
÷(x, y)
```

The quotient from Euclidean division. Computes `x/y`, truncated to an integer.

```jldoctest
julia> 9 ÷ 4
2

julia> -5 ÷ 3
-1
```


In [3]:
?/

search: / // ./ .//



```
/(x, y)
```

Right division operator: multiplication of `x` by the inverse of `y` on the right. Gives floating-point results for integer arguments.


In [4]:
7 / 2    # THIS IS NOT THE SAME AS 7 ÷ 2 - that is integer division

3.5

## Implicit Multiplication
Julia supports "implicit multiplication". So you can write code that is more math like: instead of 3*x you can just write 3x.

In [5]:
x = 5

5

In [6]:
3x

15

In [7]:
3 * x   # this works with spaces around the operator

15

In [8]:
3 x  # this doesn't - implicit multiplication requires NO SPACE

LoadError: [91msyntax: extra token "x" after end of expression[39m

In [9]:
3.5x   # how about a fraction?

17.5

## Looking at Generated Code

This one really blows my mind. (That is common). Got this from Christopher Rackauckas' blog post http://www.stochasticlifestyle.com/type-dispatch-design-post-object-oriented-programming-julia/ and had to try it.

In [10]:
foo_func(x) = x^2

foo_func (generic function with 1 method)

In [11]:
@code_llvm foo_func(1)


define i64 @julia_foo_func_63248(i64) #0 !dbg !5 {
top:
  %1 = mul i64 %0, %0
  ret i64 %1
}


In [12]:
@code_llvm foo_func(1.0)


define double @julia_foo_func_63252(double) #0 !dbg !5 {
top:
  %1 = fmul double %0, %0
  ret double %1
}


The machine code for foo_func differs for 1 (an i64) and 1.0 (a double) and is "performant" - just a single LLVM instruction!

Here's a function with 2 arguments, and all 4 combinations of i64 and double types for those arguments. Even though it is 1 function, julia creates a function for each combination of input types as needed!

In [13]:
foo_func_17(x,y) = 12*x + y + 10

foo_func_17 (generic function with 1 method)

In [14]:
@code_llvm foo_func_17(1,1)   # two integer arguments


define i64 @julia_foo_func_17_63262(i64, i64) #0 !dbg !5 {
top:
  %2 = mul i64 %0, 12
  %3 = add i64 %1, 10
  %4 = add i64 %3, %2
  ret i64 %4
}


In [15]:
@code_llvm foo_func_17(1.0,1.0)  # two double arguments


define double @julia_foo_func_17_63263(double, double) #0 !dbg !5 {
top:
  %2 = fmul double %0, 1.200000e+01
  %3 = fadd double %2, %1
  %4 = fadd double %3, 1.000000e+01
  ret double %4
}


In [16]:
@code_llvm foo_func_17(1.0,1)  # double, integer


define double @julia_foo_func_17_63265(double, i64) #0 !dbg !5 {
top:
  %2 = fmul double %0, 1.200000e+01
  %3 = sitofp i64 %1 to double
  %4 = fadd double %2, %3
  %5 = fadd double %4, 1.000000e+01
  ret double %5
}


In [17]:
@code_llvm foo_func_17(1,1.0)  # integer, double


define double @julia_foo_func_17_63267(i64, double) #0 !dbg !5 {
top:
  %2 = mul i64 %0, 12
  %3 = sitofp i64 %2 to double
  %4 = fadd double %3, %1
  %5 = fadd double %4, 1.000000e+01
  ret double %5
}


In [18]:
?@code_llvm

```
@code_llvm
```

Evaluates the arguments to the function or macro call, determines their types, and calls [`code_llvm`](@ref) on the resulting expression.


In [19]:
?code_llvm

search: code_llvm @code_llvm



```
code_llvm([io], f, types)
```

Prints the LLVM bitcodes generated for running the method matching the given generic function and type signature to `io` which defaults to `STDOUT`.

All metadata and dbg.* calls are removed from the printed bitcode. Use code_llvm_raw for the full IR.


In [20]:
?code_typed

search: code_typed @code_typed code_warntype @code_warntype



```
code_typed(f, types; optimize=true)
```

Returns an array of lowered and type-inferred ASTs for the methods matching the given generic function and type signature. The keyword argument `optimize` controls whether additional optimizations, such as inlining, are also applied.


In [21]:
@code_typed foo_func_17(1,1)

CodeInfo(:(begin 
        return (Base.add_int)((Base.add_int)((Base.mul_int)(12, x)::Int64, y)::Int64, 10)::Int64
    end))=>Int64

## Uniform Scaling Operator

In [22]:
?I

search: I IO if Int is in im Inf isa Int8 inv IPv6 IPv4 Int64 Int32 Int16 info



```
I
```

An object of type `UniformScaling`, representing an identity matrix of any size.

# Example

```jldoctest
julia> ones(5, 6) * I == ones(5, 6)
true

julia> [1 2im 3; 1im 2 3] * I
2×3 Array{Complex{Int64},2}:
 1+0im  0+2im  3+0im
 0+1im  2+0im  3+0im
```


In [23]:
I[0,0]

1

In [24]:
I[0,1]

0

Contrast with "eye" which constructs an array. "I" is extremely memory efficient by comparison.

In [25]:
?eye

search: eye KeyError speye eltype keytype supertype code_typed @code_typed



```
eye([T::Type=Float64,] m::Integer, n::Integer)
```

`m`-by-`n` identity matrix. The default element type is [`Float64`](@ref).

# Examples

```jldoctest
julia> eye(3, 4)
3×4 Array{Float64,2}:
 1.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0

julia> eye(2, 2)
2×2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0

julia> eye(Int, 2, 2)
2×2 Array{Int64,2}:
 1  0
 0  1
```

```
eye(m, n)
```

`m`-by-`n` identity matrix.

```
eye([T::Type=Float64,] n::Integer)
```

`n`-by-`n` identity matrix. The default element type is [`Float64`](@ref).

# Examples

```jldoctest
julia> eye(Int, 2)
2×2 Array{Int64,2}:
 1  0
 0  1

julia> eye(2)
2×2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0
```

```
eye(A)
```

Constructs an identity matrix of the same dimensions and type as `A`.

```jldoctest
julia> A = [1 2 3; 4 5 6; 7 8 9]
3×3 Array{Int64,2}:
 1  2  3
 4  5  6
 7  8  9

julia> eye(A)
3×3 Array{Int64,2}:
 1  0  0
 0  1  0
 0  0  1
```

Note the difference from [`ones`](@ref).


# "bits"

Thanks to https://discourse.julialang.org/u/dlfivefifty for the tip. Bits returns a string representation of all the bits of a number. This is great for teaching. (Note, the name is changing to bitstring.)

In [26]:
bits(17)

"0000000000000000000000000000000000000000000000000000000000010001"

In [27]:
bits(Float32(17.0))

"01000001100010000000000000000000"

In [28]:
bits(Float32(17.0*2))

"01000010000010000000000000000000"

In [29]:
bits(Int8(17))

"00010001"

In [30]:
bits(Int8(17*2))

"00100010"

## local scope

In [31]:
for i = 1:10
    x_for_this_loop_only=10
end
display(x_for_this_loop_only)   # an error because it is not in scope any more!

LoadError: [91mUndefVarError: x_for_this_loop_only not defined[39m

In [32]:
x_outside_loop=0  # if you want it outside the loop, you need to declare it
for i = 1:10
    x_outside_loop+=i
end
display(x_outside_loop)

55

## vec is free - it just returns a reference

In [33]:
x = [1 2 3 4]
xv = vec(x)
xv[3]=17
x

1×4 Array{Int64,2}:
 1  2  17  4

## Julia has "goto"

In [34]:
using Formatting

In [35]:
function loop_using_goto()
    i = 0
    @label L10
    printfmtln("i = {}", i)
    i += 1
    if i>=5
        @goto L20
    end
    @goto L10
    @label L20
    printfmtln("done!  i = {}", i)
end

loop_using_goto (generic function with 1 method)

In [36]:
loop_using_goto()

i = 0
i = 1
i = 2
i = 3
i = 4
done!  i = 5
